In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "/content/fra.txt"

# latent dimensionality of the encoding space
#  refers to the number of dimensions or size of the vector representation that the data is compressed into by the encoder

# the latent dimensionality of the encoding space determines the number of dimensions
# in which the input data is represented in the compressed form by the encoder.
# It is an important parameter that affects the trade-off
# between information preservation and model complexity in dimensionality reduction and representation learning tasks.

In [6]:
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()

with open(data_path, 'r') as f:
  lines = f.read().split('\n')

In [7]:
# for line in lines[: min(num_samples, len(lines) - 1)]:
# This iterates over the lines in the dataset,
# but the loop is limited to num_samples iterations
# or the number of lines in the dataset (whichever is smaller).

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_chars:
            input_chars.add(char)
    for char in target_text:
        if char not in target_chars:
            target_chars.add(char)

input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))



We will be performing one hot encoding

In [8]:
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)

# determines the maximum number of time steps the encoder RNN will process during training.
max_encoder_seq_length = max([len(txt) for txt in input_texts])
# determines the maximum number of time steps the decoder RNN will process during training
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [9]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 14
Max sequence length for outputs: 59


In [ ]:
target_texts

Performing tokenization manually

In [12]:
input_token_index = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_index = dict([(char, i) for i, char in enumerate(target_chars)])

In [ ]:
input_token_index

In [14]:
# encoder_input_data is a 3D array of shape
#  (num_pairs, max_english_sentence_length, num_english_characters)
#  containing a one-hot vectorization of the English sentences.
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)

# decoder_input_data is a 3D array of shape
# (num_pairs, max_french_sentence_length, num_french_characters)
# containg a one-hot vectorization of the French sentences.
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
# decoder_target_data is the same as decoder_input_data but offset by one timestep.
# decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :].
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

Doing one hot representation

In [15]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Building our model

First the encoder model

In [16]:
# The shape of the input is (None, num_encoder_tokens),
# where None indicates that the input sequence length can vary
# and num_encoder_tokens represents the number of unique characters in the input sequences (vocabulary size).
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))

# This line creates an LSTM layer for the encoder.
# The latent_dim parameter specifies the number of units (neurons) in the LSTM layer.
# The return_state=True argument indicates that the
# LSTM layer should return the hidden state and cell state along with the output sequence.
# This is required for capturing the encoder's final state
encoder = keras.layers.LSTM(latent_dim, return_state=True)

# The encoder_outputs variable holds the output sequence of the LSTM layer
# state_h and state_c are the hidden state and cell state
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Decoder model

In [17]:
# Seting up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# The return_sequences=True argument indicates that the LSTM layer
# should return the full output sequence, not just the final time step's output
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)

# This line applies the LSTM layer to the decoder_inputs, taking the encoder states (encoder_states) as the initial state.
# The decoder_outputs variable holds the output sequence of the LSTM layer,
# which represents the decoder's hidden state at each time step.
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# This line creates a dense layer with a softmax activation function for the decoder.
# The num_decoder_tokens parameter specifies the number of units (neurons) in the dense layer,
#  which is equal to the number of unique characters in the target sequences (vocabulary size).
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")

# This line applies the dense layer to the decoder_outputs,
# transforming the LSTM's hidden state at each time step
# into a probability distribution over the target characters
decoder_outputs = decoder_dense(decoder_outputs)

In [18]:
# Defining the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [19]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

Training the model

In [22]:
history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=100,
    validation_split=0.2,
)
model.save("content/s2s")

Epoch 1/100
125/125 [==============================] - 2s 14ms/step - loss: 0.5293 - accuracy: 0.8443 - val_loss: 0.5868 - val_accuracy: 0.8300
Epoch 2/100
125/125 [==============================] - 2s 12ms/step - loss: 0.5133 - accuracy: 0.8492 - val_loss: 0.5702 - val_accuracy: 0.8365
Epoch 3/100
125/125 [==============================] - 1s 11ms/step - loss: 0.4995 - accuracy: 0.8539 - val_loss: 0.5577 - val_accuracy: 0.8386
Epoch 4/100
125/125 [==============================] - 1s 12ms/step - loss: 0.4856 - accuracy: 0.8581 - val_loss: 0.5423 - val_accuracy: 0.8435
Epoch 5/100
125/125 [==============================] - 1s 11ms/step - loss: 0.4731 - accuracy: 0.8618 - val_loss: 0.5356 - val_accuracy: 0.8442
Epoch 6/100
125/125 [==============================] - 2s 14ms/step - loss: 0.4628 - accuracy: 0.8642 - val_loss: 0.5314 - val_accuracy: 0.8455
Epoch 7/100
125/125 [==============================] - 1s 12ms/step - loss: 0.4532 - accuracy: 0.8664 - val_loss: 0.5155 - val_accuracy:

Defining sampling models

Restoring the model and constructing the encoder and decoder.

In [23]:
# loads pretrained s2s model
model = keras.models.load_model("content/s2s")

# Restoring the encoder model
encoder_inputs = model.input[0]  # retrieves the encoder input layer from the loaded model.
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
# creates the encoder model using the keras.Model class
# with encoder_inputs as input and encoder_states as output.
encoder_model = keras.Model(encoder_inputs, encoder_states)

# Restoring the decoder model

decoder_inputs = model.input[1] # retrieves the decoder input layer from the loaded model.
# define input layers for the initial decoder states (hidden state and cell state).
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
# creates a list containing the input layers for the initial decoder states
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3] #retrieves the decoder LSTM layer from the loaded model.
# applies the decoder LSTM to the decoder input, using the initial decoder states as the initial state.
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
# creates a list containing the updated decoder states.
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4] #retrieves the decoder dense layer (output layer) from the loaded model.
# applies the decoder dense layer to the decoder LSTM's output
decoder_outputs = decoder_dense(decoder_outputs)
# creates the decoder model using the keras.Model class.
# The input layers are [decoder_inputs] + decoder_states_inputs,
# and the output layers are [decoder_outputs] + decoder_states.
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)



In [24]:
# Reverse-lookup token index to decode sequences back to
# something readable.
# The code creates dictionaries that map the integer indices
# back to the characters in the input and target vocabularies.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

 Creating a function that generates a decoded sentence (output sequence) given an input sequence.

function decode_sequence(input_seq) perform sequence-to-sequence decoding using the trained encoder and decoder models

In [26]:
def decode_sequence(input_seq):
    # encodes the input sequence input_seq using the encoder_model,
    # which returns the initial hidden state and cell state (states_value) of the decoder.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""

    # The while loop iteratively generates the decoded sentence until a stop condition is met.
    # The stop conditions are either encountering the "end sequence" character ("\n")
    # or reaching the maximum allowed length (max_decoder_seq_length)
    while not stop_condition:
        # predicts the output probability distribution (output_tokens)
        #  and updates the decoder's hidden state (h) and cell state (c) using the decoder_model.
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [30]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index+1000 : seq_index + 1001]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index+1000])
    print("Decoded sentence:", decoded_sentence)

1/1 [==============================] - 0s 22ms/step
-
Input sentence: Grab this.
Decoded sentence: Attrapez ça !

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Hands off.
Decoded sentence: Tenez-vous à la mien !

1/1 [==============================] - 0s 22ms/step
-
Input sentence: Hands off.
Decoded sentence: Tenez-vous à la mien !

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Have some.
Decoded sentence: Prends-en.

1/1 [==============================] - 0s 22ms/step
-
Input sentence: Have some.
Decoded sentence: Prends-en.

1/1 [==============================] - 0s 22ms/step
-
Input sentence: He is ill.
Decoded sentence: Il est paresseux.

1/1 [==============================] - 0s 27ms/step
-
Input sentence: He is old.
Decoded sentence: Il est paresseux.

1/1 [==============================] - 0s 21ms/step
-
Input sentence: He's a DJ.
Decoded sentence: C'est un bizante.

1/1 [==============================] - 0s 36ms/step
-
Input sent